In [1]:
# Cell 1
import os
import getpass

# 1. Groq API 
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter Groq API Key: ")

# 2. LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "AI-Tutor-Groq"

if "LANGCHAIN_API_KEY" not in os.environ:
    key_input = getpass.getpass("Enter LangSmith API Key: ")
    if key_input:
        os.environ["LANGCHAIN_API_KEY"] = key_input

print("Environment Configured successfully.")

Enter Groq API Key:  ········
Enter LangSmith API Key:  ········


Environment Configured successfully.


In [2]:
# Cell 2

import os
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
warnings.filterwarnings('ignore')
logging.getLogger('tensorflow').setLevel(logging.ERROR)

from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.tools import DuckDuckGoSearchRun
from pydantic import BaseModel, Field
from typing import List, TypedDict, Optional
from langchain_core.documents import Document

# MODEL CONFIGURATION
MODEL_NAME = "llama-3.3-70b-versatile"

llm = ChatGroq(
    model=MODEL_NAME,
    temperature=0.3,
    max_retries=2,
)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
web_search = DuckDuckGoSearchRun()

# DATA SCHEMA
class Checkpoint(BaseModel):
    id: int = Field(description="The checkpoint number (1-5)")
    topic: str = Field(description="The main title of the sub-topic")
    objective: str = Field(description="A concise learning objective for this checkpoint")

class StudyPlan(BaseModel):
    topic: str = Field(description="The main subject requested by the user")
    checkpoints: List[Checkpoint] = Field(description="List of 5 generated learning checkpoints")

# STATE DEFINITION
class AgentState(TypedDict):
    main_topic: str
    file_path: Optional[str]
    study_plan: List[dict]
    current_checkpoint: dict
    topic: str
    objective: str

    # Document & Context Storage
    documents: List[Document]
    doc_context: str
    web_context: str
    
    # Logic Flags
    source: str
    validation_status: str
    needs_web_search: bool

    # Quiz & Adaptation Logic
    quiz_questions: List[str]
    user_answers: List[str]
    quiz_results: List[dict]    
    failed_concepts: List[str]  
    quiz_score: int            

    # Output Storage
    final_essay: str
    quiz_feedback: str

    # Retry & Evaluation
    retries: int
    best_score: int
    best_essay: str

print(f"Model Configured: {MODEL_NAME} (via Groq)")


Model Configured: llama-3.3-70b-versatile (via Groq)


In [3]:
# Cell 3: Curriculum Planning Node
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import PydanticOutputParser

def plan_curriculum(state: AgentState):
    print(f"GENERATING STUDY PLAN FOR: {state['main_topic']}")
    
    parser = PydanticOutputParser(pydantic_object=StudyPlan)
    
    # Check if we have a document context loaded in the state
    doc_context = state.get("doc_context", "")
    context_instruction = ""
    
    # Only verify doc usage if actual text exists
    if doc_context and len(doc_context) > 10: 
        context_instruction = (
            "CONTEXT FROM USER DOCUMENT:\n"
            f"{doc_context[:1500]}\n" 
            "INSTRUCTION: Base the checkpoints strictly on the structure/chapters "
            "provided in the context above."
        )
    else:
        context_instruction = "Generate the plan based on general web knowledge."

    prompt = f"""
    You are an expert Curriculum Designer.
    Create a structured 5-part study plan for the topic: "{state['main_topic']}".
    
    {context_instruction}
    
    REQUIREMENTS:
    1. Break the topic into exactly 5 logical progression steps (Checkpoints).
    2. For each checkpoint, provide a Title and a specific Learning Objective.
    3. Ensure the difficulty increases from Checkpoint 1 to 5.
    
    FORMAT INSTRUCTION:
    {parser.get_format_instructions()}
    """
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        parsed_plan = parser.parse(response.content)
        
        # Convert Pydantic models to regular dicts for state storage
        plan_dicts = [cp.dict() for cp in parsed_plan.checkpoints]
        return {"study_plan": plan_dicts}

    except Exception as e:
        print(f"Error generating plan: {e}")
        # Fallback Plan if LLM fails
        return {"study_plan": [
            {"id": 1, "topic": f"{state['main_topic']} Basics", "objective": "Introduction to core concepts"},
            {"id": 2, "topic": "Key Mechanisms", "objective": "Understanding how it works"},
            {"id": 3, "topic": "Advanced Concepts", "objective": "Deep dive into complexities"},
            {"id": 4, "topic": "Applications", "objective": "Real-world use cases"},
            {"id": 5, "topic": "Current State & Future", "objective": "State of the art analysis"}
        ]}

In [4]:
# Cell 4
import os
import warnings

warnings.filterwarnings("ignore", message=".*The `dict` method is deprecated.*")

from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document

# 1. GET USER INPUT
main_topic_input = input("What do you want to learn? ")

# DETECT AND LOAD DOCUMENT
documents_loaded = []
full_text = "" 
file_input = None

if os.path.exists("notes.pdf"):
    file_input = "notes.pdf"
    print(f"Document detected: {file_input}")
    
    try:
        print("Loading content into memory...")
        loader = PyPDFLoader(file_input)
        pages = loader.load()
        
        full_text = "\n\n".join([p.page_content for p in pages])
        
        documents_loaded = [Document(page_content=full_text, metadata={"source": "user_upload"})]
        print(f"Successfully loaded {len(full_text)} characters.")
        
    except Exception as e:
        print(f"Error reading PDF: {e}")
else:
    print("No 'notes.pdf' found. Proceeding without document.")

initial_state = {
    "main_topic": main_topic_input,
    "file_path": file_input, 
    "documents": documents_loaded, 
    
    "study_plan": [],
    "current_checkpoint": {},
    "topic": "", 
    "objective": "", 
    
    "doc_context": full_text, 
    
    "web_context": "", 
    "final_essay": "", 
    "quiz_questions": [], 
    "user_answers": [], 
    "quiz_score": 0, 
    "quiz_feedback": "",
    "needs_web_search": False, 
    "retries": 0, 
    "best_score": 0, 
    "best_essay": "",
    "source": "", 
    "validation_status": "",
    "failed_concepts": []
}

# 2. GENERATE THE PLAN
print("\nConsulting Curriculum Designer...")
plan_result = plan_curriculum(initial_state)
study_plan = plan_result["study_plan"]

# 3. DISPLAY & SELECT
print(f"\nSTUDY PLAN FOR: {main_topic_input.upper()}")
for cp in study_plan:
    print(f"{cp['id']}. {cp['topic']}")
    print(f"   Objective: {cp['objective']}")

while True:
    try:
        choice = int(input("Select a Checkpoint to start (1-5): "))
        if 1 <= choice <= 5:
            selected_cp = study_plan[choice-1]
            print(f"\nSelected: {selected_cp['topic']}")
            
            # UPDATE STATE FOR THE REST OF THE APP
            initial_state["topic"] = selected_cp["topic"]
            initial_state["objective"] = selected_cp["objective"]
            initial_state["study_plan"] = study_plan
            initial_state["current_checkpoint"] = selected_cp
            break
        print("Invalid choice. Please enter 1-5.")
    except ValueError:
        print("Please enter a number.")

What do you want to learn?  deep learning


Document detected: notes.pdf
Loading content into memory...
Successfully loaded 3312 characters.

Consulting Curriculum Designer...
GENERATING STUDY PLAN FOR: deep learning

STUDY PLAN FOR: DEEP LEARNING
1. Introduction to Artificial Neural Networks
   Objective: Understand the basic architecture and components of Artificial Neural Networks, including input, hidden, and output layers.
2. Forward and Backward Propagation in ANNs
   Objective: Learn how forward propagation calculates the output and backward propagation updates weights to minimize loss in Artificial Neural Networks.
3. Introduction to Convolutional Neural Networks
   Objective: Comprehend the basic structure and key components of Convolutional Neural Networks, including convolutional layers and their application to grid-like data.
4. Advanced Concepts in Convolutional Neural Networks
   Objective: Delve deeper into the specifics of Convolutional Neural Networks, including the role of filters, kernels, and the preservation

Select a Checkpoint to start (1-5):  3



Selected: Introduction to Convolutional Neural Networks


In [5]:
# Cell 5
import json
import re
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage

def check_user_doc_and_grade(state):
    topic = state.get('topic', 'Unknown')
    objective = state.get('objective', 'General understanding')
    documents = state.get("documents", [])
    
    print(f"CHECKING USER DOCUMENT FOR TOPIC: {topic}")

    # 1. INSTANT FAIL: No documents uploaded
    if not documents:
        print("No user document found. Proceeding to Web Search.")
        return {
            "source": "web_search", 
            "validation_status": "needs_search"
        }

    # 2. PREPARE CONTENT FOR GRADING
    doc_snippet = documents[0].page_content[:3000]
    
    # 3. LLM GRADING PROMPT
    prompt = f"""
    You are a Teacher's Assistant. Grade the relevance of the following document excerpt.
    
    STUDENT TOPIC: "{topic}"
    LEARNING OBJECTIVE: "{objective}"
    
    DOCUMENT EXCERPT:
    {doc_snippet}...
    
    TASK:
    Determine if this document contains SUFFICIENT information to teach the topic.
    
    CRITERIA:
    - SUFFICIENT: The document covers the topic well. No web search needed.
    - INCOMPLETE: The document mentions the topic but lacks detail/depth. Needs web search to fill gaps.
    - IRRELEVANT: The document is about something else entirely. Needs full web search.
    
    Return JSON ONLY: {{"status": "SUFFICIENT" | "INCOMPLETE" | "IRRELEVANT"}}
    """
    
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        
        # Parse JSON output
        json_match = re.search(r"\{.*\}", response.content, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group(0))
            status = data.get("status", "IRRELEVANT").upper()
        else:
            status = "IRRELEVANT"
            
    except Exception as e:
        print(f"   -> Grading Error: {e}. Defaulting to Search.")
        status = "IRRELEVANT"

    # 4. DECISION LOGIC
    print(f"Document Grade: {status}")

    if status == "SUFFICIENT":
        print("Document is sufficient. Skipping Web Search.")
        return {
            "source": "user_doc", 
            "validation_status": "relevant"
        }
        
    elif status == "INCOMPLETE":
        print("Document is incomplete. Will use Web Search to supplement.")
        # 'mixed' source tells the generator to use both PDF and Web results
        return {
            "source": "mixed", 
            "validation_status": "needs_search"
        }
        
    else: # IRRELEVANT
        print("Document is irrelevant. Will perform fresh Web Search.")
        return {
            "source": "web_search", 
            "validation_status": "needs_search"
        }

In [6]:
# Cell 6: Web Search
def perform_web_search(state):
    topic = state.get('topic', 'Unknown')
    current_source = state.get('source', 'web_search')
    print(f"SEARCHING WEB FOR: {topic} (Mode: {current_source})")
    
    try:
        # Perform the search
        search_result_text = web_search.invoke(topic)
        print("Search completed successfully.")
        
    except Exception as e:
        print(f"Search failed: {e}")
        search_result_text = "Web search failed or returned no results."

    return {"web_context": search_result_text}

In [7]:
# Cell 7: Generate Essay
from langchain_core.messages import HumanMessage

def generate_essay(state):
    topic = state.get("topic")
    objective = state.get("objective")
    source_mode = state.get("source", "web_search")
    
    # 1. CONTEXT SELECTION LOGIC
    user_text = state.get("doc_context", "")
    web_text = state.get("web_context", "")
    final_context = ""

    if source_mode == "user_doc":
        print("GENERATOR: Using strictly User Document.")
        final_context = f"SOURCE MATERIAL (User Notes):\n{user_text[:15000]}" 
        
    elif source_mode == "web_search":
        print("GENERATOR: Using strictly Web Search.")
        final_context = f"SOURCE MATERIAL (Web Search):\n{web_text}"
        
    elif source_mode == "mixed":
        print("GENERATOR: Using Hybrid Context (User Notes + Web).")
        final_context = (
            f"PRIMARY SOURCE (User Notes):\n{user_text[:10000]}\n\n"
            f"SUPPLEMENTARY SOURCE (Web Search):\n{web_text}"
        )
    
    # 2. GENERATE ESSAY
    print(f"GENERATING ESSAY FOR: {topic}")
    essay_prompt = f"""
    You are an expert tutor teaching: "{topic}"
    Objective: "{objective}"
    
    INSTRUCTIONS:
    Write a comprehensive, clear, and engaging educational guide (approx 400-600 words).
    Base your teaching STRICTLY on the Source Material provided below.
    If the User Notes are unclear, use the Web Search context to clarify, but prioritize user notes.
    
    {final_context}
    """
    
    try:
        essay_response = llm.invoke([HumanMessage(content=essay_prompt)])
        essay_content = essay_response.content
    except Exception as e:
        print(f"Essay Gen Error: {e}")
        essay_content = "Error generating content."

    return {
        "final_essay": essay_content
    }

In [8]:
# Cell 8
import re
import json
from langchain_core.messages import HumanMessage

def grade_essay(state):
    print("GRADING ESSAY AGAINST OBJECTIVE")
    
    essay = state.get("final_essay", "")
    topic = state.get("topic", "")
    objective = state.get("objective", "")
    current_retries = state.get("retries", 0)
    best_score_so_far = state.get("best_score", 0)
    
    if "Error generating essay" in essay or len(essay) < 100:
        print("Generation failed or extremely short.")
        return {
            "best_score": best_score_so_far,
            "retries": current_retries + 1,
        }

    # Validator Prompt
    prompt = f"""
    You are a Quality Control Editor.
    
    TARGET TOPIC: {topic}
    REQUIRED OBJECTIVE: {objective}
    
    DRAFT LESSON:
    {essay[:2000]}... [truncated]
    
    EVALUATION CHECKLIST (0-5 Score):
    1. Does it explain the specific TOPIC? (+1)
    2. Does it meet the specific OBJECTIVE? (+1)
    3. Is it FREE of complex math notation (No LaTeX/Formulas)? (+1)
    4. Is it clear and beginner-friendly? (+1)
    5. Is the depth and length appropriate for the topic's complexity? (+1)
    
    Return JSON ONLY: {{"score": int, "reasoning": "string"}}
    """
    
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        # Parse JSON
        json_match = re.search(r"\{.*\}", response.content, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group(0))
            score = data.get("score", 0)
            reason = data.get("reasoning", "No reasoning provided.")
        else:
            score = 3
            reason = "Could not parse validation JSON."
            
    except Exception as e:
        print(f"Validation error: {e}")
        score = 3
        reason = f"Exception: {e}"

    print(f"Score: {score}/5 | Reason: {reason}")

    updates = {
        "retries": current_retries + 1
    }
    
    if score >= best_score_so_far:
        updates["best_score"] = score
        updates["best_essay"] = essay
        print("New best essay saved.")
    
    return updates

In [9]:
# Cell 9: Graph Routing & Logic Edges

def route_source_check(state):
    """
    EDGE LOGIC: Determines path after Cell 5 (Relevance Check).
    Reads 'validation_status' set by the Router.
    """
    status = state.get("validation_status", "needs_search")
    print(f"ROUTING DECISION: {status}")
    
    if status == "relevant":
        return "skip_search"  
    else:
        return "perform_search"

def check_quality_gate(state):
    """
    EDGE LOGIC: Determines path after Cell 8 (Grading).
    Checks if essay needs refinement or is ready for Quiz.
    """
    score = state.get("best_score", 0)
    retries = state.get("retries", 0)
    
    print(f"QUALITY CHECK: Score={score}, Retries={retries}")

    # 1. Success (High Score)
    if score >= 4:
        print(">> QUALITY PASS. Proceeding to Quiz.")
        return "pass"
    
    # 2. Max Retries Reached 
    if retries >= 3:
        print(">> MAX RETRIES. Proceeding with current draft.")
        return "pass"
    
    # 3. Fail (Low Score)
    print(">> QUALITY FAIL. Retrying Generation.")
    return "retry"

def finalize_submission(state):
    """
    NODE: Final cleanup. Ensures 'final_essay' holds the best version.
    """
    print("FINALIZING CHECKPOINT...")
    best = state.get("best_essay")
    current = state.get("final_essay")
    
    final = best if best and len(best) > len(current) else current
    
    return {"final_essay": final}

In [10]:
# Cell 10: Compile Generator Graph
from langgraph.graph import StateGraph, END

# Initialize Graph
workflow = StateGraph(AgentState)

# 1. ADD NODES
workflow.add_node("check_user_doc", check_user_doc_and_grade) 
workflow.add_node("perform_web_search", perform_web_search)  
workflow.add_node("generate_essay", generate_essay)            
workflow.add_node("grade_essay", grade_essay) 
workflow.add_node("finalize_submission", finalize_submission) 


# 2. DEFINE FLOW

workflow.set_entry_point("check_user_doc")

def route_source_check(state):
    return state.get("validation_status", "needs_search")

workflow.add_conditional_edges(
    "check_user_doc",
    route_source_check,
    {
        "relevant": "generate_essay",        
        "needs_search": "perform_web_search" 
    }
)

workflow.add_edge("perform_web_search", "generate_essay")

workflow.add_edge("generate_essay", "grade_essay")

# Retry Loop
workflow.add_conditional_edges(
    "grade_essay",
    check_quality_gate, 
    {
        "pass": "finalize_submission", 
        "retry": "generate_essay"     
    }
)

workflow.add_edge("finalize_submission", END)

# Compile the graph
app = workflow.compile()
print("Dynamic Study Graph Compiled Successfully.")

Dynamic Study Graph Compiled Successfully.


In [11]:
# Cell 11: Execute Selected Checkpoint

if 'initial_state' not in globals() or not initial_state.get("topic"):
    print("ERROR: No checkpoint selected. Please Run Cell 4 first to generate a plan.")
else:
    print(f"EXECUTING LESSON PLAN: {initial_state['topic']}")
    print(f"OBJECTIVE: {initial_state['objective']}")
    
    # Run the Graph
    result = app.invoke(initial_state)
    
    # Display Result
    print(f"LESSON COMPLETE: {result.get('topic', 'Unknown Topic')}")
    
    print(f"QUALITY SCORE: {result.get('best_score', 0)}/5") 
    
    print(result.get("final_essay", "No essay generated."))

EXECUTING LESSON PLAN: Introduction to Convolutional Neural Networks
OBJECTIVE: Comprehend the basic structure and key components of Convolutional Neural Networks, including convolutional layers and their application to grid-like data.
CHECKING USER DOCUMENT FOR TOPIC: Introduction to Convolutional Neural Networks
Document Grade: SUFFICIENT
Document is sufficient. Skipping Web Search.
GENERATOR: Using strictly User Document.
GENERATING ESSAY FOR: Introduction to Convolutional Neural Networks
GRADING ESSAY AGAINST OBJECTIVE
Score: 5/5 | Reason: The draft lesson effectively explains the topic of Convolutional Neural Networks, meeting the specific objective of comprehending the basic structure and key components of CNNs. It is free of complex math notation, making it accessible to beginners. The language used is clear and beginner-friendly, providing a concise introduction to the subject. The depth and length of the lesson are appropriate for the topic's complexity, covering the essential

In [12]:
# Cell 12: Context Processing Node 

def process_context(state: AgentState):
    print("PROCESSING CONTEXT FOR QUIZ...")
    
    # 1. PRIORITY: USE GENERATED CONTENT
    if state.get("final_essay") and len(state["final_essay"]) > 100:
        print("Using GENERATED ESSAY as the source for questions.")
        
        return {
            "doc_context": state["final_essay"], 
            "needs_web_search": False
        }
    else:
        print("No essay found. Logic Error. Triggering Fallback.")
        return {"needs_web_search": True, "doc_context": ""}

In [13]:
# Cell 13: Strict Context-Based Quiz Generator (Targeted Re-Quiz)
import json
import re
from langchain_core.messages import HumanMessage

def generate_quiz(state):
    topic = state.get("topic")
    failed_concepts = state.get("failed_concepts", [])
    
    # 1. GET CONTENT
    context_text = (state.get("final_essay") or "").strip()
    
    if len(context_text) < 100:
        context_text = (state.get("doc_context") or "").strip()

    print(f"QUIZ GENERATOR CONTEXT LENGTH: {len(context_text)} chars ---")
    
    # 2. PREPARE PROMPT
    if failed_concepts:
        print(f"Generating 5 Targeted Questions for Weak Areas: {failed_concepts}")
        
        prompt = f"""
        You are a strict Exam Setter.
        
        SOURCE TEXT (Contains Original Lesson + Remedial Explanations):
        {context_text[:8000]}
        
        TASK:
        The student failed specific concepts: {failed_concepts}.
        Generate 5 new conceptual questions that focus **EXCLUSIVELY** on these weak concepts.
        
        STRICT RULES:
        1. **Scope:** Ask questions ONLY about {failed_concepts}. Do not ask about topics the student already mastered.
        2. **Synthesis:** Use BOTH the original technical content AND the "Remedial Explanation" analogies in the source text to frame your questions.
        3. **Variety:** If there is only 1 weak concept, ask 5 different questions testing that single concept from different angles (definition, application, analogy, comparison).
        4. **Source:** Answers must be derivable from the Source Text provided.
        
        FORMAT: Return a JSON List of strings: ["Question 1", "Question 2", "Question 3", "Question 4", "Question 5"]
        """
    else:
        # CASE B: FRESH QUIZ
        prompt = f"""
        You are a strict Exam Setter.
        
        SOURCE TEXT:
        {context_text[:6000]}
        
        TASK:
        Generate 5 conceptual questions based EXCLUSIVELY on the Source Text above.
        
        STRICT RULES:
        1. Every question MUST be answerable using strictly the provided text.
        2. Do NOT ask generic questions like "What is {topic}?". Ask specific details found in the text.
        3. Cover 5 distinct sub-topics found in the text.
        
        FORMAT: Return a JSON List of strings: ["Question 1", "Question 2", "Question 3", "Question 4", "Question 5"]
        """

    # 3. EXECUTE
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        
        json_match = re.search(r"\[.*\]", response.content, re.DOTALL)
        if json_match:
            questions = json.loads(json_match.group(0))
        else:
            questions = [f"Based on the text, explain {topic}."]
            
        return {"quiz_questions": questions}

    except Exception as e:
        print(f"Quiz Gen Error: {e}")
        return {"quiz_questions": [f"What is {topic}?"]}

In [14]:
# Cell 14: Strict Quiz Grading Node 

import json
import re
from langchain_core.messages import HumanMessage

def grade_quiz(state):
    print("GRADING QUIZ ANSWERS...")
    
    questions = state.get("quiz_questions", [])
    answers = state.get("user_answers", [])
    topic = state.get("topic", "")
    
    # Combine Q & A for the LLM
    qa_pairs = []
    for i, (q, a) in enumerate(zip(questions, answers)):
        qa_pairs.append(f"Q{i+1}: {q}\nA{i+1}: {a}")
    
    qa_text = "\n\n".join(qa_pairs)
    
    prompt = f"""
    You are a strict Academic Examiner.
    Topic: {topic}
    
    Evaluate these 5 Question-Answer pairs.
    
    TASK:
    1. Assign a score out of 20 for EACH answer.
    2. Provide a brief FEEDBACK for each answer and ways to improve that answer.
    
    STRICT GRADING RULES:
    - 20 Marks: Perfect. Comprehensive, accurate, and uses correct terminology.
    - 15-19 Marks: Strong answer. Concept is correct, but has minor phrasing issues or misses a small nuance/example.
    - 10-14 Marks: Partially correct. Core concept is visible but missing key details or terminology.
    - 1-9 Marks: Weak. Contains significant errors, vague guessing, or major omissions.
    - 0 Marks: Completely incorrect, irrelevant, "I don't know", or just repeating the question.
    
    Do NOT give points for effort. Be objective.
    
    INPUT:
    {qa_text}
    
    OUTPUT FORMAT (JSON ONLY):
    {{
        "results": [
            {{
                "question_index": 1, 
                "score": 0, 
                "feedback": "Feedback text here...",
                "concept_topic": "Topic Name"
            }},
            ...
        ]
    }}
    """
    
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        
        # Parse JSON
        json_match = re.search(r"\{.*\}", response.content, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group(0))
            results = data.get("results", [])
        else:
            results = [{"score": 0, "feedback": "Error parsing", "concept_topic": "Unknown"} for _ in range(5)]

        # ANALYZE SCORES
        total_score_raw = 0
        failed_concepts = []
        
        for res in results:
            score = res.get("score", 0)
            total_score_raw += score
            
            if score < 15:
                concept = res.get("concept_topic", "General Concept")
                failed_concepts.append(concept)
        
        final_percentage = int(total_score_raw) 
        
        return {
            "quiz_results": results,
            "quiz_score": final_percentage, 
            "failed_concepts": failed_concepts
        }

    except Exception as e:
        print(f"Grading Error: {e}")
        return {"quiz_score": 0, "failed_concepts": ["Error in grading"]}

In [15]:
# Cell 15
def check_pass_fail(state: AgentState):
    score = state["quiz_score"]
    
    if score >= 70:
        print("PASSED (>70). Evaluation Successful.")
        return "pass"
    else:
        print("FAILED (<70). Routing to Remedial Node.")
        return "fail"

In [16]:
# Cell 16: Feynman Remedial Node 
from langchain_core.messages import HumanMessage

def generate_feynman_explanation(state):
    print("\nFEYNMAN MODE ACTIVATED (Score < 70%)")
    
    topic = state.get("topic", "")
    
    # 1. RETRIEVE FAILED CONCEPTS
    failed_concepts = state.get("failed_concepts", [])
    
    if not failed_concepts:
        failed_concepts = [f"Core concepts of {topic}"]

    print(f"Feynman is preparing a lesson on: {failed_concepts}")

    # 2. FEYNMAN PROMPT
    prompt = f"""
    You are Richard Feynman, the Great Explainer. 
    The student is struggling to understand the following specific concepts related to "{topic}":
    
    WEAK CONCEPTS:
    {failed_concepts}
    
    YOUR TASK:
    1. Ignore previous quiz questions. Focus on explaining these CONCEPTS from scratch.
    2. Use **SIMPLE ANALOGIES** and plain English (e.g., comparing data structures to trains, bookshelves, buckets, traffic).
    3. Be encouraging, but ensure the technical depth is sufficient to answer exam questions.
    4. Explain it clearly and concisely (maximum 500 words).
    
    Write the remedial explanation now.
    """
    
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        feynman_text = response.content
        
        print("\nGenerated Remedial Explanation.")
        
        return {
            "feynman_explanation": feynman_text,
            "final_essay": state.get("final_essay", "") 
        }
        
    except Exception as e:
        print(f"Feynman Node Error: {e}")
        return {"final_essay": state.get("final_essay", "")}

In [17]:
# Cell 17: Interactive Assessment Graph
from langgraph.graph import StateGraph, END

# Initialize Graph
workflow_v2 = StateGraph(AgentState)

# 1. ADD NODES
workflow_v2.add_node("process_context", process_context)
workflow_v2.add_node("perform_web_search", perform_web_search)
workflow_v2.add_node("generate_quiz", generate_quiz)

# 2. DEFINE EDGES
workflow_v2.set_entry_point("process_context")

def check_search_requirement(state):
    return "perform_web_search" if state.get("needs_web_search") else "generate_quiz"

workflow_v2.add_conditional_edges(
    "process_context",
    check_search_requirement,
    {
        "perform_web_search": "perform_web_search",
        "generate_quiz": "generate_quiz"
    }
)

workflow_v2.add_edge("perform_web_search", "generate_quiz")

workflow_v2.add_edge("generate_quiz", END)

# Compile
app_v2 = workflow_v2.compile()
print("Interactive Assessment Graph Compiled Successfully.")

Interactive Assessment Graph Compiled Successfully.


In [18]:
# Cell 18: Interactive Quiz Loop

# 1. SETUP: VERIFY CONTENT
if 'result' not in globals() or not result.get('final_essay'):
    print("ERROR: No generated lesson found.")
    print("Please run Cell 11 first to generate the study material.")
else:
    # 2. LOAD INITIAL CONTENT
    current_topic = result.get('topic', initial_state['topic'])
    original_essay = result.get('final_essay', "")
    
    print(f"STARTING QUIZ FOR: {current_topic}")

    # 3. INITIALIZE STATE
    loop_state = {
        "topic": current_topic,
        "final_essay": original_essay, 
        "quiz_questions": [], 
        "user_answers": [], 
        "quiz_score": 0,
        "failed_concepts": [],
        "doc_context": original_essay, 
        "needs_web_search": False,
        "quiz_results": []
    }

    # 4. START LOOP
    max_feynman_triggers = 2
    attempts = 0

    while attempts <= max_feynman_triggers:
        attempts += 1
        print(f"\nQUIZ ATTEMPT {attempts}")
        
        # A. GENERATE QUESTIONS
        print("(Generating Questions based on current context...)")
        quiz_output = app_v2.invoke(loop_state)
        questions = quiz_output.get('quiz_questions', [])
        loop_state['quiz_questions'] = questions

        # B. ASK QUESTIONS
        user_answers = []
        if questions:
            for i, q in enumerate(questions, 1):
                print(f"\nQ{i}: {q}")
                ans = input(f"Answer: ")
                user_answers.append(ans)
        else:
            print("Error: No questions generated.")
            break

        # C. GRADE ANSWERS
        print("\nGrading Answers...")
        loop_state["user_answers"] = user_answers
        
        grading_output = grade_quiz(loop_state)
        
        total_score = grading_output.get('quiz_score', 0)
        current_failed_concepts = grading_output.get('failed_concepts', [])
        results_detail = grading_output.get('quiz_results', [])
        
        # DISPLAY DETAILED FEEDBACK
        print("DETAILED FEEDBACK")
        
        for i, res in enumerate(results_detail):
            q_num = res.get("question_index", i+1)
            q_score = res.get("score", 0)
            q_feedback = res.get("feedback", "No feedback provided.")
            
            print(f"Q{q_num}: {q_score}/20")
            print(f"   Feedback: {q_feedback}\n")
            
        print(f"TOTAL SCORE: {total_score}%")
        
        # D. PASS CHECK
        if total_score >= 70:
            print(f"\nPASSED! You have mastered {current_topic}.")
            print("Checkpoint Complete.")
            break
            
        # E. FAIL CHECK
        else:
            print(f"FAILED (<70%). Weak Areas: {current_failed_concepts}")
            
            if attempts <= max_feynman_triggers:
                print(f"\nTriggering Feynman Node ({attempts}/{max_feynman_triggers})...")
                
                feynman_input = {
                    "topic": current_topic,
                    "quiz_results": results_detail, 
                    "user_answers": user_answers,
                    "quiz_questions": questions,
                    "final_essay": original_essay
                }
                
                if 'generate_feynman_explanation' in globals():
                    feynman_output = generate_feynman_explanation(feynman_input)
                    explanation = feynman_output.get("feynman_explanation", "")
                    
                    print("\nFEYNMAN EXPLANATION:")
                    print(explanation)
                    
                    # F. UPDATE CONTEXT
                    combined_context = (
                        f"{original_essay}\n\n"
                        f"=== REMEDIAL EXPLANATION ===\n"
                        f"{explanation}"
                    )
                    
                    loop_state["final_essay"] = combined_context
                    loop_state["failed_concepts"] = current_failed_concepts
                    
                    input("\nPress Enter to take the Re-Quiz...")
                else:
                    print("Error: Feynman module not found.")
                    break
            else:
                print("\nMAXIMUM ATTEMPTS REACHED.")
                print(f"You have triggered the Feynman remedial help {max_feynman_triggers} times and still scored below 70%.")
                print("ACTION REQUIRED: You require additional help to clear this checkpoint.")
                break

STARTING QUIZ FOR: Introduction to Convolutional Neural Networks

QUIZ ATTEMPT 1
(Generating Questions based on current context...)
PROCESSING CONTEXT FOR QUIZ...
Using GENERATED ESSAY as the source for questions.
QUIZ GENERATOR CONTEXT LENGTH: 2718 chars ---

Q1: What are the three main types of layers in an Artificial Neural Network (ANN)?


KeyboardInterrupt: Interrupted by user